<center> <h1>Rocket Control</h1> </center>
<center> <h2>Réalisation : Habibi Issam - Sajid Badr</h2> </center>

**Originally Contributed by**: Iain Dunning

This tutorial shows how to solve a nonlinear rocketry control problem.
The problem was drawn from the [COPS3](http://www.mcs.anl.gov/~more/cops/cops3.pdf) benchmark.

Our goal is to maximize the final altitude of a vertically launched rocket. 
We can control the thrust of the rocket, and must take account of 
the rocket mass, fuel consumption rate, gravity, and aerodynamic drag.

Let us consider the basic description of the model (for the full description, 
including parameters for the rocket, see the COPS3 PDF)

### Overview 
We will use a discretized model of time, with a fixed number of time steps, $n$. 
We will make the time step size $\Delta t$, and thus the final time $t_f = n \cdot \Delta t$, a variable in the problem. 
To approximate the derivatives in the problem we will use the [trapezoidal rule](http://en.wikipedia.org/wiki/Trapezoidal_rule).

### State and Control 
We will have three state variables:

* Velocity, $v$
* Altitude, $h$
* Mass of rocket and remaining fuel, $m$

and a single control variable, thrust $T$. 
Our goal is thus to maximize $h(t_f)$. 
Each of these corresponds to a JuMP variable indexed by the time step.

### Dynamics
We have three equations that control the dynamics of the rocket:

Rate of ascent: $$h^\prime = v$$
Acceleration: $$v^\prime = \frac{T - D(h,v)}{m} - g(h)$$
Rate of mass loss: $$m^\prime = -\frac{T}{c}$$

where drag $D(h,v)$ is a function of altitude and velocity, and gravity $g(h)$ is a function of altitude.

These forces are defined as

$$D(h,v) = D_c v^2 exp\left( -h_c \left( \frac{h-h(0)}{h(0)} \right) \right)$$
and
$$g(h) = g_0 \left( \frac{h(0)}{h} \right)^2$$

The three rate equations correspond to JuMP constraints, 
and for convenience we will represent the forces with nonlinear expressions.

In [2]:
import Pkg; Pkg.add("Ipopt")
using JuMP, Ipopt
# Can visualize the state and control variables
using Gadfly

   Updating registry at `~/.julia/registries/General`
######################################################################### 100,0%
  Resolving package versions...
  Installed METIS_jll ───── v5.1.0+5
  Installed Ipopt_jll ───── v3.13.4+0
  Installed MUMPS_seq_jll ─ v5.2.1+4
  Installed ASL_jll ─────── v0.1.2+0
  Installed Qt_jll ──────── v5.15.2+3
  Installed Ipopt ───────── v0.6.5
  Installed MathProgBase ── v0.7.8
Updating `~/.julia/environments/v1.5/Project.toml`
  [b6b21f68] + Ipopt v0.6.5
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [ae81ac8f] + ASL_jll v0.1.2+0
  [b6b21f68] + Ipopt v0.6.5
  [9cc047cb] + Ipopt_jll v3.13.4+0
  [d00139f3] + METIS_jll v5.1.0+5
  [d7ed1dd3] + MUMPS_seq_jll v5.2.1+4
  [fdba3010] + MathProgBase v0.7.8
  [ede63266] ↑ Qt_jll v5.15.2+2 ⇒ v5.15.2+3
   Building Ipopt → `~/.julia/packages/Ipopt/P1XLY/deps/build.log`
┌ Info: Precompiling Ipopt [b6b21f68-93f8-5de0-b562-5493be1d77c9]
└ @ Base loading.jl:1278


LoadError: [91mArgumentError: Package Gadfly not found in current path:[39m
[91m- Run `import Pkg; Pkg.add("Gadfly")` to install the Gadfly package.[39m


<div class="alert alert-info">

**_Work:_**
    
1. Execute the code and determine the structure of the optimal control.
2. Add a constraint $ v(t) \le 0.1$ and re-execute the code. What have changed?
3. Play on the number of steps $n$ and on the discretization (Trapezoidal vs Rectangular integration). 
      
</div>

In [ ]:
# Create JuMP model, using Ipopt as the solver
rocket = Model(optimizer_with_attributes(Ipopt.Optimizer, "print_frequency_iter" => 1))

# Constants
# Note that all parameters in the model have been normalized
# to be dimensionless. See the COPS3 paper for more info.
h_0 = 1    # Initial height
v_0 = 0    # Initial velocity
m_0 = 1    # Initial mass
g_0 = 1    # Gravity at the surface

T_c = 3.5  # Used for thrust
h_c = 500  # Used for drag
v_c = 620  # Used for drag
m_c = 0.6  # Fraction of initial mass left at end

c     = 0.5 * sqrt(g_0 * h_0)  # Thrust-to-fuel mass
m_f   = m_c * m_0              # Final mass
D_c   = 0.5 * v_c * m_0 / g_0  # Drag scaling
T_max = T_c * g_0 * m_0        # Maximum thrust

n = 800   # Time steps

@variables(rocket, begin
    Δt ≥ 0, (start = 1/n) # Time step
    # State variables
    v[1:n] ≥ 0            # Velocity
    h[1:n] ≥ h_0          # Height
    m_f ≤ m[1:n] ≤ m_0    # Mass
    # Control
    0 ≤ T[1:n] ≤ T_max    # Thrust
end)

# Objective: maximize altitude at end of time of flight
@objective(rocket, Max, h[n])

# Initial conditions
@constraints(rocket, begin
    v[1] == v_0
    h[1] == h_0
    m[1] == m_0
    m[n] == m_f
end)

# Forces
# Drag(h,v) = Dc v^2 exp( -hc * (h - h0) / h0 )
@NLexpression(rocket, drag[j = 1:n], D_c * (v[j]^2) * exp(-h_c * (h[j] - h_0) / h_0))
# Grav(h)   = go * (h0 / h)^2
@NLexpression(rocket, grav[j = 1:n], g_0 * (h_0 / h[j])^2)
# Time of flight
@NLexpression(rocket, t_f, Δt * n)

# Dynamics
for j in 2:n
    # h' = v
    
    # Rectangular integration
    # @NLconstraint(rocket, h[j] == h[j - 1] + Δt * v[j - 1])
    
    # Trapezoidal integration
    @NLconstraint(rocket,
        h[j] == h[j - 1] + 0.5 * Δt * (v[j] + v[j - 1]))

    # v' = (T-D(h,v))/m - g(h)
    
    # Rectangular integration
    # @NLconstraint(rocket, v[j] == v[j - 1] + Δt *(
    #                 (T[j - 1] - drag[j - 1]) / m[j - 1] - grav[j - 1]))
    
    # Trapezoidal integration
    @NLconstraint(rocket,
        v[j] == v[j-1] + 0.5 * Δt * (
            (T[j] - drag[j] - m[j] * grav[j]) / m[j] +
            (T[j - 1] - drag[j - 1] - m[j - 1] * grav[j - 1]) / m[j - 1]))

    # m' = -T/c

    # Rectangular integration
    # @NLconstraint(rocket, m[j] == m[j - 1] - Δt * T[j - 1] / c)
    
    # Trapezoidal integration
    @NLconstraint(rocket,
        m[j] == m[j - 1] - 0.5 * Δt * (T[j] + T[j-1]) / c)
end

# Solve for the control and state
println("Solving...")
status = optimize!(rocket)

# Display results
# println("Solver status: ", status)
println("Max height: ", objective_value(rocket))

In [ ]:
h_plot = plot(x = (1:n) * value.(Δt), y = value.(h)[:], Geom.line,
                Guide.xlabel("Time (s)"), Guide.ylabel("Altitude"))
m_plot = plot(x = (1:n) * value.(Δt), y = value.(m)[:], Geom.line,
                Guide.xlabel("Time (s)"), Guide.ylabel("Mass"))
v_plot = plot(x = (1:n) * value.(Δt), y = value.(v)[:], Geom.line,
                Guide.xlabel("Time (s)"), Guide.ylabel("Velocity"))
T_plot = plot(x = (1:n) * value.(Δt), y = value.(T)[:], Geom.line,
                Guide.xlabel("Time (s)"), Guide.ylabel("Thrust"))
draw(SVG(6inch, 6inch), vstack(hstack(h_plot, m_plot), hstack(v_plot, T_plot)))